In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
hs2_codes =  pd.read_csv(r"C:\Users\Admin\Downloads\hs2_codes_corrected.csv")

In [3]:
hs2_codes

,HS2,Description
0,1,Live animals
1,2,Meat and edible meat offal
2,3,Fish & crustaceans
3,4,Dairy produce
4,5,Products of animal origin
...,...,...
93,95,Miscellaneous manufactured articles
94,96,"Works of art, collectors' pieces"
95,97,Special transactions
96,98,Special classification provisions


In [4]:
hs6_codes = pd.read_csv(r"C:\Users\Admin\Desktop\Exports_project\product_codes_HS17_V202501.csv")

In [5]:
hs6_codes = hs6_codes.rename(columns = {'code' : 'HS6',
                                       'description' : 'Description'})

In [6]:
hs6_codes

,HS6,Description
0,10121,"Horses: live, pure-bred breeding animals"
1,10129,"Horses: live, other than pure-bred breeding an..."
2,10130,Asses: live
3,10190,Mules and hinnies: live
4,10221,"Cattle: live, pure-bred breeding animals"
...,...,...
5379,970300,"Sculptures and statuary: original, in any mate..."
5380,970400,"Stamps, postage or revenue: stamp-postmarks, f..."
5381,970500,Collections and collectors' pieces: of zoologi...
5382,970600,Antiques: of an age exceeding one hundred years


In [7]:
hs2_codes['HS2'] = hs2_codes['HS2'].astype(str)  # Convert to string if not already

# Add '0' as prefix for 5-digit values
hs2_codes['HS2'] = hs2_codes['HS2'].apply(lambda x: '0' + x if len(x) == 1 else x)

# Assuming HS6_desc is your DataFrame and 'code' is the column with HS6 values
hs6_codes['HS6'] = hs6_codes['HS6'].astype(str)  # Convert to string if not already

# Add '0' as prefix for 5-digit values
hs6_codes['HS6'] = hs6_codes['HS6'].apply(lambda x: '0' + x if len(x) == 5 else x)

In [8]:
hs2_codes

,HS2,Description
0,01,Live animals
1,02,Meat and edible meat offal
2,03,Fish & crustaceans
3,04,Dairy produce
4,05,Products of animal origin
...,...,...
93,95,Miscellaneous manufactured articles
94,96,"Works of art, collectors' pieces"
95,97,Special transactions
96,98,Special classification provisions


In [9]:
hs6_codes

,HS6,Description
0,010121,"Horses: live, pure-bred breeding animals"
1,010129,"Horses: live, other than pure-bred breeding an..."
2,010130,Asses: live
3,010190,Mules and hinnies: live
4,010221,"Cattle: live, pure-bred breeding animals"
...,...,...
5379,970300,"Sculptures and statuary: original, in any mate..."
5380,970400,"Stamps, postage or revenue: stamp-postmarks, f..."
5381,970500,Collections and collectors' pieces: of zoologi...
5382,970600,Antiques: of an age exceeding one hundred years


In [10]:
df_dict = {}
for file in os.listdir(os.getcwd()):
    if file.startswith('BACI'):
        filename = file.split('_')
        print(filename[2])
        
        df = pd.read_csv(file, dtype={"k": str})
        df = df.rename(columns={
            't': 'year',
            'i': 'exporter',
            'j': 'importer',
            'k': 'HS6',
            'v': 'value(thousands USD)',
            'q': 'quantity(in metric tons)'
        })
        
        # Ensure 6-digit product codes
        df["HS6"] = df["HS6"].str.zfill(6)

        # Extract HS2 and HS4 codes
        df["HS2"] = df["HS6"].str[:2]  # First 2 digits
        df["HS4"] = df["HS6"].str[:4]

        # Merge with HS2 and HS6 description data
        df = df.merge(hs2_codes, on="HS2", how="left").rename(columns={"Description": "HS2_desc"})
        df = df.merge(hs6_codes, on="HS6", how="left").rename(columns={"Description": "HS6_desc"})
        
        df_dict[filename[2]] = df  # Store in dictionary with year as key

Y2017
Y2018
Y2019
Y2020
Y2021
Y2022
Y2023


In [24]:
df_dict['Y2017'][(df_dict['Y2017']['exporter'] == 156) & (df_dict['Y2017']['importer'] == 842)]

,year,exporter,importer,HS6,value(thousands USD),quantity(in metric tons),HS2,HS4,HS2_desc,HS6_desc
1775471,2017,156,842,010129,910.276,41.240,01,0101,Live animals,"Horses: live, other than pure-bred breeding an..."
1775472,2017,156,842,010611,32834.903,44.283,01,0106,Live animals,"Mammals: live, primates"
1775473,2017,156,842,010612,140.000,1.307,01,0106,Live animals,"Mammals: live, whales, dolphins and porpoises ..."
1775474,2017,156,842,010619,119.262,0.587,01,0106,Live animals,"Mammals: live, other than primates, whales, do..."
1775475,2017,156,842,010620,14.003,0.157,01,0106,Live animals,Reptiles: live (including snakes and turtles)
...,...,...,...,...,...,...,...,...,...,...
1780161,2017,156,842,970200,1051.572,1.145,97,9702,Special transactions,"Engravings, prints and lithographs: original"
1780162,2017,156,842,970300,30367.043,NaN,97,9703,Special transactions,"Sculptures and statuary: original, in any mate..."
1780163,2017,156,842,970400,641.883,4.449,97,9704,Special transactions,"Stamps, postage or revenue: stamp-postmarks, f..."
1780164,2017,156,842,970500,42798.468,835.332,97,9705,Special transactions,Collections and collectors' pieces: of zoologi...


In [22]:
for key in df_dict.keys():
    print(len(df_dict[key][(df_dict[key]['exporter'] == 156) & (df_dict[key]['importer'] == 842)]))

4695
4724
4669
4619
4629
4668
4619


In [25]:
#master df

In [39]:
master_df.to_csv('filtered_data.csv',index=False)

# report again only for china to usa

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
master_df = pd.read_csv('filtered_data.csv')

In [5]:
master_df

,year,exporter,importer,HS6,value(thousands USD),quantity(in metric tons),HS2,HS4,HS2_desc,HS6_desc
0,2017,156,842,10129,910,42,1,101,Live animals,"Horses: live, other than pure-bred breeding an..."
1,2017,156,842,10611,32834,45,1,106,Live animals,"Mammals: live, primates"
2,2017,156,842,10612,140,2,1,106,Live animals,"Mammals: live, whales, dolphins and porpoises ..."
3,2017,156,842,10619,119,1,1,106,Live animals,"Mammals: live, other than primates, whales, do..."
4,2017,156,842,10620,14,1,1,106,Live animals,Reptiles: live (including snakes and turtles)
...,...,...,...,...,...,...,...,...,...,...
32618,2023,156,842,970200,485,1,97,9702,Special transactions,"Engravings, prints and lithographs: original"
32619,2023,156,842,970300,15690,875,97,9703,Special transactions,"Sculptures and statuary: original, in any mate..."
32620,2023,156,842,970400,28,1,97,9704,Special transactions,"Stamps, postage or revenue: stamp-postmarks, f..."
32621,2023,156,842,970500,3855,5,97,9705,Special transactions,Collections and collectors' pieces: of zoologi...


In [7]:
master_df['HS2'] = master_df['HS2'].astype(str).str.zfill(2)
master_df['HS4'] = master_df['HS4'].astype(str).str.zfill(4)
master_df['HS6'] = master_df['HS6'].astype(str).str.zfill(6)

In [8]:
master_df

,year,exporter,importer,HS6,value(thousands USD),quantity(in metric tons),HS2,HS4,HS2_desc,HS6_desc
0,2017,156,842,010129,910,42,01,0101,Live animals,"Horses: live, other than pure-bred breeding an..."
1,2017,156,842,010611,32834,45,01,0106,Live animals,"Mammals: live, primates"
2,2017,156,842,010612,140,2,01,0106,Live animals,"Mammals: live, whales, dolphins and porpoises ..."
3,2017,156,842,010619,119,1,01,0106,Live animals,"Mammals: live, other than primates, whales, do..."
4,2017,156,842,010620,14,1,01,0106,Live animals,Reptiles: live (including snakes and turtles)
...,...,...,...,...,...,...,...,...,...,...
32618,2023,156,842,970200,485,1,97,9702,Special transactions,"Engravings, prints and lithographs: original"
32619,2023,156,842,970300,15690,875,97,9703,Special transactions,"Sculptures and statuary: original, in any mate..."
32620,2023,156,842,970400,28,1,97,9704,Special transactions,"Stamps, postage or revenue: stamp-postmarks, f..."
32621,2023,156,842,970500,3855,5,97,9705,Special transactions,Collections and collectors' pieces: of zoologi...


In [9]:
desired_order = [
    'year',
    'exporter',
    'importer',
    'HS2',
    'HS4',
    'HS6',  # moved here
    'value(thousands USD)',
    'quantity(in metric tons)',
    'HS2_desc',
    'HS6_desc'
]

master_df = master_df[desired_order]

In [11]:
del master_df['exporter']
del master_df['importer']

In [12]:
master_df

,year,HS2,HS4,HS6,value(thousands USD),quantity(in metric tons),HS2_desc,HS6_desc
0,2017,01,0101,010129,910,42,Live animals,"Horses: live, other than pure-bred breeding an..."
1,2017,01,0106,010611,32834,45,Live animals,"Mammals: live, primates"
2,2017,01,0106,010612,140,2,Live animals,"Mammals: live, whales, dolphins and porpoises ..."
3,2017,01,0106,010619,119,1,Live animals,"Mammals: live, other than primates, whales, do..."
4,2017,01,0106,010620,14,1,Live animals,Reptiles: live (including snakes and turtles)
...,...,...,...,...,...,...,...,...
32618,2023,97,9702,970200,485,1,Special transactions,"Engravings, prints and lithographs: original"
32619,2023,97,9703,970300,15690,875,Special transactions,"Sculptures and statuary: original, in any mate..."
32620,2023,97,9704,970400,28,1,Special transactions,"Stamps, postage or revenue: stamp-postmarks, f..."
32621,2023,97,9705,970500,3855,5,Special transactions,Collections and collectors' pieces: of zoologi...


In [13]:
need_hs=['04', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '17', '18', '19', '20', '21', '22', '23', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40']

In [16]:
need_master_df = master_df[master_df['HS2'].isin(need_hs)]

In [22]:
for col in need_master_df.columns:
    if col == 'HS2':
        for code in need_master_df[col].unique():
            need_master_df['value(thousands USD)'][need_master_df[col] == code].sum()

year
HS2
HS4
HS6
value(thousands USD)
quantity(in metric tons)
HS2_desc
HS6_desc


In [28]:
need_master_df['value(thousands USD)'][need_master_df['HS2'] == '04'].sum()

121752

In [53]:
top_hs2

Index(['39', '29', '40', '30', '38', '28', '33', '20', '15', '21'], dtype='object', name='HS2')

In [35]:
from collections import defaultdict

# Initialize final dictionary
final_dict = defaultdict(lambda: {"HS2": {}, "HS4": {}})

# Process each year
for year in need_master_df['year'].unique():
    year_df = need_master_df[need_master_df['year'] == year]

    # Get top 10 HS2 codes
    top_hs2 = (
        year_df.groupby('HS2')['value(thousands USD)']
        .sum()
        .sort_values(ascending=False)
        .head(10)
        .index
    )

    for hs2 in top_hs2:
        hs2_df = year_df[year_df['HS2'] == hs2]

        # Add HS2 summary
        final_dict[year]["HS2"][hs2] = {
            "total_value": hs2_df['value(thousands USD)'].sum(),
            "total_quantity": hs2_df['quantity(in metric tons)'].sum()
        }

        # Get top 5 HS4 under HS2
        top_hs4 = (
            hs2_df.groupby('HS4')['value(thousands USD)']
            .sum()
            .sort_values(ascending=False)
            .head(5)
            .index
        )

        # Store list of top 5 HS4 with their values
        final_dict[year]["HS4"][hs2] = []

        for hs4 in top_hs4:
            hs4_df = hs2_df[hs2_df['HS4'] == hs4]

            final_dict[year]["HS4"][hs2].append({
                "HS4_code": hs4,
                "total_value": hs4_df['value(thousands USD)'].sum(),
                "total_quantity": hs4_df['quantity(in metric tons)'].sum()
            })

In [36]:
final_dict.keys()

dict_keys([2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [37]:
final_dict

defaultdict(<function __main__.<lambda>()>,
            {2017: {'HS2': {'39': {'total_value': 16937297,
                'total_quantity': 3972312},
               '29': {'total_value': 7463032, 'total_quantity': 1367064},
               '40': {'total_value': 3216533, 'total_quantity': 689873},
               '33': {'total_value': 1489950, 'total_quantity': 135349},
               '30': {'total_value': 1431386, 'total_quantity': 72818},
               '28': {'total_value': 1416070, 'total_quantity': 1027148},
               '20': {'total_value': 1249453, 'total_quantity': 909300},
               '38': {'total_value': 1213215, 'total_quantity': 305399},
               '27': {'total_value': 830491, 'total_quantity': 889674},
               '07': {'total_value': 756543, 'total_quantity': 284415}},
              'HS4': {'39': [{'HS4_code': '3926',
                 'total_value': 4777923,
                 'total_quantity': 443276},
                {'HS4_code': '3924',
                 'total

In [40]:
need_master_df

,year,HS2,HS4,HS6,value(thousands USD),quantity(in metric tons),HS2_desc,HS6_desc
132,2017,04,0402,040229,0,1,Dairy produce,"Dairy produce: milk and cream, containing adde..."
133,2017,04,0402,040299,302,221,Dairy produce,"Dairy produce: milk and cream, containing adde..."
134,2017,04,0405,040510,97,15,Dairy produce,"Dairy produce: derived from milk, butter"
135,2017,04,0407,040729,271,148,Dairy produce,"Birds' eggs, in shell: fresh, not for incubati..."
136,2017,04,0407,040790,5480,1649,Dairy produce,"Birds' eggs, in shell: preserved or cooked"
...,...,...,...,...,...,...,...,...
29582,2023,40,4016,401693,243373,21075,Rubber & articles thereof,"Rubber: vulcanised (other than hard rubber), g..."
29583,2023,40,4016,401694,5122,1561,Rubber & articles thereof,"Rubber: vulcanised (other than hard rubber), b..."
29584,2023,40,4016,401695,6365,800,Rubber & articles thereof,"Rubber: vulcanised (other than hard rubber), i..."
29585,2023,40,4016,401699,436210,90738,Rubber & articles thereof,"Rubber: vulcanised (other than hard rubber), a..."


In [51]:
need_master_df['value(thousands USD)'][(need_master_df['HS2'] == '39') & (need_master_df['year'] == 2021)].sum()

26723581

In [52]:
need_master_df['quantity(in metric tons)'][(need_master_df['HS2'] == '39') & (need_master_df['year'] == 2021)].sum()

6036213